In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
cd gdrive/My Drive/TFM/

/content/gdrive/My Drive/TFM


In [ ]:
pip install dask[dataframe] --upgrade

     |████████████████████████████████| 132 kB 5.3 MB/s 


In [ ]:

import sys
import time
import datetime as dt
import multiprocessing as mp
from datetime import datetime
import pandas as pd

import re
import os

import pandas as pd
import pandas_datareader.data as web
from pandas import Timestamp

import numpy as np
import scipy.stats as stats
import statsmodels.api as sm

import math



from itertools import cycle
from scipy import interp

# import visual tools
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline
import seaborn as sns
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from google.colab import drive

import glob

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


#aux methods

In [ ]:
#SNIPPET 3.1 DAILY VOLATILITY ESTIMATES

def get_daily_vol(close, lookback=100):
    """
    :param close: (data frame) Closing prices
    :param lookback: (int) lookback period to compute volatility
    :return: (series) of daily volatility value
    """
    print('Calculating daily volatility for dynamic thresholds')
    
    df0 = close.index.searchsorted(close.index - pd.Timedelta(days=1))
    df0 = df0[df0 > 0]
    df0 = (pd.Series(close.index[df0 - 1], index=close.index[close.shape[0] - df0.shape[0]:]))
        
    df0 = close.loc[df0.index] / close.loc[df0.values].values - 1  # daily returns
    df0 = df0.ewm(span=lookback).std()
    return df0

In [ ]:
def corr_metric(corr, use_abs=False):
    corr = _fix_corr(corr)
    if use_abs:
        return np.sqrt(1 - np.abs(corr))
    else:
        return np.sqrt(0.5 * (1 - corr))

In [ ]:
#Serial correlation aka auto correlation

def returns(candles_close_prices):
    return pd.Series(np.diff(np.log(candles_close_prices)))

#Loading the data in this case with all symbols for a autocorrelation of different sampling pricedures. Experiment with Volume and Dollar bars for different time frames.

In [ ]:

column_names = ['Volume-1','Volume-2','Volume-3','Volume-4','Volume-5','Dollar-1','Dollar-2', 'Dollar-3','Dollar-4', 'Dollar-5' ]
row_names = ['ADAUSDT','BNBUSDT','EOSUSDT','DOTUSDT','ETHUSD','LINKUSDT','LTCUSD','XTZUSDT','XBTUSD','XRPUSD']

def bar(xs, y): return np.int64(xs / y) * y

correlations = pd.DataFrame(columns = column_names,index = row_names )

In [ ]:
# raw trade data from https://public.bitmex.com/?prefix=data/trade/ 
data = pd.DataFrame()
for i,file in enumerate(glob.glob("data/test/Data_0/*.csv")):
  if i == 0:
    data = data.append(pd.read_csv(file))
    print('Percentge of files already Loaded:',round((i/len(glob.glob("data/test/Data_0/*.csv")))*100,1), '%. There are', len(glob.glob("data/test/Data_0/*.csv"))-i, "files left", end='')
  else:
    data = data.append(pd.read_csv(file))
    print('\r Percentge of files already Loaded:',round((i/len(glob.glob("data/test/Data_0/*.csv")))*100,1), '%. There are', len(glob.glob("data/test/Data_0/*.csv"))-i, "files left",end='', flush=True)

In [ ]:
# raw trade data from https://public.bitmex.com/?prefix=data/trade/ 
data = pd.DataFrame()
for i,file in enumerate(glob.glob("data/test/Data_1/*.csv")):
  if i == 0:
    data = data.append(pd.read_csv(file))
    print('Percentge of files already Loaded:',round((i/len(glob.glob("data/test/Data_1/*.csv")))*100,1), '%. There are', len(glob.glob("data/test/Data_1/*.csv"))-i, "files left", end='')
  else:
    data = data.append(pd.read_csv(file))
    print('\r Percentge of files already Loaded:',round((i/len(glob.glob("data/test/Data_1/*.csv")))*100,1), '%. There are', len(glob.glob("data/test/Data_1/*.csv"))-i, "files left",end='', flush=True)

In [ ]:
# raw trade data from https://public.bitmex.com/?prefix=data/trade/ 
data = pd.DataFrame()
for i,file in enumerate(glob.glob("data/test/Data_2/*.csv")):
  if i == 0:
    data = data.append(pd.read_csv(file))
    print('Percentge of files already Loaded:',round((i/len(glob.glob("data/test/Data_2/*.csv")))*100,1), '%. There are', len(glob.glob("data/test/Data_2/*.csv"))-i, "files left", end='')
  else:
    data = data.append(pd.read_csv(file))
    print('\r Percentge of files already Loaded:',round((i/len(glob.glob("data/test/Data_2/*.csv")))*100,1), '%. There are', len(glob.glob("data/test/Data_2/*.csv"))-i, "files left",end='', flush=True)

In [ ]:
# raw trade data from https://public.bitmex.com/?prefix=data/trade/ 
data = pd.DataFrame()
for i,file in enumerate(glob.glob("data/test/Data_3/*.csv")):
  if i == 0:
    data = data.append(pd.read_csv(file))
    print('Percentge of files already Loaded:',round((i/len(glob.glob("data/test/Data_3/*.csv")))*100,1), '%. There are', len(glob.glob("data/test/Data_3/*.csv"))-i, "files left", end='')
  else:
    data = data.append(pd.read_csv(file))
    print('\r Percentge of files already Loaded:',round((i/len(glob.glob("data/test/Data_3/*.csv")))*100,1), '%. There are', len(glob.glob("data/test/Data_3/*.csv"))-i, "files left",end='', flush=True)

##ADA

In [ ]:
data1 = data[data.symbol == 'ADAUSDTH21']
#del(data)
# timestamp parsing
data1['timestamp'] = data1.timestamp.map(lambda t: datetime.strptime(t[:-3], "%Y-%m-%dD%H:%M:%S.%f"))

In [ ]:
Time_bars_1d = data1.groupby(pd.Grouper(key="timestamp", freq="1d")).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first', 'foreignNotional':'sum' })
Time_bars_1d.columns = ['open','high','low','close','size','timestamp','foreignNotional']
avg_dly_dollar = np.mean(Time_bars_1d['foreignNotional'])

avg_15m_dollar = np.round(avg_dly_dollar/96,decimals=-3)

avg_30m_dollar = np.round(avg_dly_dollar/48, decimals = -3)

avg_1h_dollar = np.round(avg_dly_dollar/24,decimals=-2)

avg_4h_dollar = np.round(avg_dly_dollar/5,decimals=-2)

avg_12h_dollar = np.round(avg_dly_dollar/2,decimals=-4)

avg_dollars = np.array([avg_15m_dollar, avg_30m_dollar, avg_1h_dollar, avg_4h_dollar, avg_12h_dollar])



Time_bars_1d = data1.groupby(pd.Grouper(key="timestamp", freq="1d")).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first' })
Time_bars_1d.columns = ['open','high','low','close','size','timestamp']
avg_dly_volume = np.mean(Time_bars_1d['size'])

avg_15m_volume = np.round(avg_dly_volume/96,decimals=-1)

avg_30m_volume = np.round(avg_dly_volume/48, decimals = -1)

avg_1h_volume = np.round(avg_dly_volume/24,decimals=-2)

avg_4h_volume = np.round(avg_dly_volume/5,decimals=-2)

avg_12h_volume = np.round(avg_dly_volume/2,decimals=-2)

avg_volumes = np.array([avg_15m_volume, avg_30m_volume, avg_1h_volume, avg_4h_volume, avg_12h_volume])



print(avg_volumes,'\n',avg_dollars)

[  60.  130.  300. 1200. 3000.] 
 [  37000.   75000.  149200.  716200. 1790000.]


In [ ]:
Volume_bars_50 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[0])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_50.columns = ['open','high','low','close','size','timestamp']
correlations.at['ADAUSDT','Volume-1'] = returns(Volume_bars_50['close']).autocorr(lag=1)


Volume_bars_100 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[1])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_100.columns = ['open','high','low','close','size','timestamp']
correlations.at['ADAUSDT','Volume-2'] = returns(Volume_bars_100['close']).autocorr(lag=1)


Volume_bars_500 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[2])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_500.columns = ['open','high','low','close','size','timestamp']
correlations.at['ADAUSDT','Volume-3'] = returns(Volume_bars_500['close']).autocorr(lag=1)


Volume_bars_1000 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[3])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_1000.columns = ['open','high','low','close','size','timestamp']
correlations.at['ADAUSDT','Volume-4'] = returns(Volume_bars_1000['close']).autocorr(lag=1)


Volume_bars_5000 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[4])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_5000.columns = ['open','high','low','close','size','timestamp']
correlations.at['ADAUSDT','Volume-5'] = returns(Volume_bars_5000['close']).autocorr(lag=1)



Dollar_bars_500 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[0])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_500.columns = ['open','high','low','close','size','timestamp']
correlations.at['ADAUSDT','Dollar-1'] = returns(Dollar_bars_500['close']).autocorr(lag=1)


Dollar_bars_1000 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[1])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_1000.columns = ['open','high','low','close','size','timestamp']
correlations.at['ADAUSDT','Dollar-2'] = returns(Dollar_bars_1000['close']).autocorr(lag=1)


Dollar_bars_10000 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[2])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_10000.columns = ['open','high','low','close','size','timestamp']
correlations.at['ADAUSDT','Dollar-3'] = returns(Dollar_bars_10000['close']).autocorr(lag=1)


Dollar_bars_100000 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[3])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_100000.columns = ['open','high','low','close','size','timestamp']
correlations.at['ADAUSDT','Dollar-4'] = returns(Dollar_bars_100000['close']).autocorr(lag=1)

Dollar_bars_1000000 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[4])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_1000000.columns = ['open','high','low','close','size','timestamp']
correlations.at['ADAUSDT','Dollar-5'] = returns(Dollar_bars_1000000['close']).autocorr(lag=1)


correlations


,Volume-1,Volume-2,Volume-3,Volume-4,Volume-5,Dollar-1,Dollar-2,Dollar-3,Dollar-4,Dollar-5
ADAUSDT,-0.0222421,-0.0162059,-0.0234384,-0.0403136,0.00702243,-0.047659,-0.00279578,0.0111431,-0.0662191,-0.0582422
BNBUSDT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EOSUSDT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DOTUSDT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ETHUSD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LINKUSDT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LTCUSD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
XTZUSDT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
XBTUSD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
XRPUSD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##BNB

In [ ]:
data1 = data[data.symbol == 'BNBUSDTH21']
del(data)
# timestamp parsing
data1['timestamp'] = data1.timestamp.map(lambda t: datetime.strptime(t[:-3], "%Y-%m-%dD%H:%M:%S.%f"))

In [ ]:
Time_bars_1d = data1.groupby(pd.Grouper(key="timestamp", freq="1d")).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first', 'foreignNotional':'sum' })
Time_bars_1d.columns = ['open','high','low','close','size','timestamp','foreignNotional']
avg_dly_dollar = np.mean(Time_bars_1d['foreignNotional'])

avg_15m_dollar = np.round(avg_dly_dollar/96,decimals=-3)

avg_30m_dollar = np.round(avg_dly_dollar/48, decimals = -3)

avg_1h_dollar = np.round(avg_dly_dollar/24,decimals=-3)

avg_4h_dollar = np.round(avg_dly_dollar/5,decimals=-4)

avg_12h_dollar = np.round(avg_dly_dollar/2,decimals=-4)

avg_dollars = np.array([avg_15m_dollar, avg_30m_dollar, avg_1h_dollar, avg_4h_dollar, avg_12h_dollar])



Time_bars_1d = data1.groupby(pd.Grouper(key="timestamp", freq="1d")).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first' })
Time_bars_1d.columns = ['open','high','low','close','size','timestamp']
avg_dly_volume = np.mean(Time_bars_1d['size'])

avg_15m_volume = np.round(avg_dly_volume/96,decimals=-1)

avg_30m_volume = np.round(avg_dly_volume/48, decimals = -1)

avg_1h_volume = np.round(avg_dly_volume/24,decimals=-2)

avg_4h_volume = np.round(avg_dly_volume/5,decimals=-3)

avg_12h_volume = np.round(avg_dly_volume/2,decimals=-3)

avg_volumes = np.array([avg_15m_volume, avg_30m_volume, avg_1h_volume, avg_4h_volume, avg_12h_volume])

print(avg_volumes,'\n',avg_dollars)

In [ ]:
Volume_bars_50 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[0])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_50.columns = ['open','high','low','close','size','timestamp']
correlations.at['BNBUSDT','Volume-1'] = returns(Volume_bars_50['close']).autocorr(lag=1)


Volume_bars_100 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[1])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_100.columns = ['open','high','low','close','size','timestamp']
correlations.at['BNBUSDT','Volume-2'] = returns(Volume_bars_100['close']).autocorr(lag=1)


Volume_bars_500 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[2])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_500.columns = ['open','high','low','close','size','timestamp']
correlations.at['BNBUSDT','Volume-3'] = returns(Volume_bars_500['close']).autocorr(lag=1)


Volume_bars_1000 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[3])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_1000.columns = ['open','high','low','close','size','timestamp']
correlations.at['BNBUSDT','Volume-4'] = returns(Volume_bars_1000['close']).autocorr(lag=1)


Volume_bars_5000 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[4])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_5000.columns = ['open','high','low','close','size','timestamp']
correlations.at['BNBUSDT','Volume-5'] = returns(Volume_bars_5000['close']).autocorr(lag=1)



Dollar_bars_500 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[0])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_500.columns = ['open','high','low','close','size','timestamp']
correlations.at['BNBUSDT','Dollar-1'] = returns(Dollar_bars_500['close']).autocorr(lag=1)


Dollar_bars_1000 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[1])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_1000.columns = ['open','high','low','close','size','timestamp']
correlations.at['BNBUSDT','Dollar-2'] = returns(Dollar_bars_1000['close']).autocorr(lag=1)


Dollar_bars_10000 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[2])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_10000.columns = ['open','high','low','close','size','timestamp']
correlations.at['BNBUSDT','Dollar-3'] = returns(Dollar_bars_10000['close']).autocorr(lag=1)


Dollar_bars_100000 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[3])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_100000.columns = ['open','high','low','close','size','timestamp']
correlations.at['BNBUSDT','Dollar-4'] = returns(Dollar_bars_100000['close']).autocorr(lag=1)

Dollar_bars_1000000 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[4])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_1000000.columns = ['open','high','low','close','size','timestamp']
correlations.at['BNBUSDT','Dollar-5'] = returns(Dollar_bars_1000000['close']).autocorr(lag=1)


correlations


,Volume-1,Volume-2,Volume-3,Volume-4,Volume-5,Dollar-1,Dollar-2,Dollar-3,Dollar-4,Dollar-5
ADAUSDT,-0.0222421,-0.0162059,-0.0234384,-0.0403136,0.00702243,-0.047659,-0.00279578,0.0111431,-0.0662191,-0.0582422
BNBUSDT,-0.00430166,-0.00774128,-0.0157777,-0.0192799,-0.0799558,-0.121679,-0.0157654,-6.21017e-06,0.0296318,-0.0154122
EOSUSDT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DOTUSDT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ETHUSD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LINKUSDT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LTCUSD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
XTZUSDT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
XBTUSD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
XRPUSD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##EOS

In [ ]:
data1 = data[data.symbol == 'EOSUSDTH21']

# timestamp parsing
data1['timestamp'] = data1.timestamp.map(lambda t: datetime.strptime(t[:-3], "%Y-%m-%dD%H:%M:%S.%f"))

In [ ]:
correlations.append(pd.Series(name='EOSUSDT'))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.



,Volume-1,Volume-2,Volume-3,Volume-4,Volume-5,Dollar-1,Dollar-2,Dollar-3,Dollar-4,Dollar-5
ADAUSDT,-0.0222421,-0.0162059,-0.0234384,-0.0403136,0.00702243,-0.047659,-0.00279578,0.0111431,-0.0662191,-0.0582422
BNBUSDT,-0.00430166,-0.00774128,-0.0157777,-0.0192799,-0.0799558,-0.121679,-0.0157654,-6.21017e-06,0.0296318,-0.0154122
EOSUSDT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DOTUSDT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ETHUSD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LINKUSDT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LTCUSD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
XTZUSDT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
XBTUSD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
XRPUSD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
Time_bars_1d = data1.groupby(pd.Grouper(key="timestamp", freq="1d")).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first', 'foreignNotional':'sum' })
Time_bars_1d.columns = ['open','high','low','close','size','timestamp','foreignNotional']
avg_dly_dollar = np.mean(Time_bars_1d['foreignNotional'])

avg_15m_dollar = np.round(avg_dly_dollar/96,decimals=-3)

avg_30m_dollar = np.round(avg_dly_dollar/48, decimals = -3)

avg_1h_dollar = np.round(avg_dly_dollar/24,decimals=-3)

avg_4h_dollar = np.round(avg_dly_dollar/5,decimals=-4)

avg_12h_dollar = np.round(avg_dly_dollar/2,decimals=-4)

avg_dollars = np.array([avg_15m_dollar, avg_30m_dollar, avg_1h_dollar, avg_4h_dollar, avg_12h_dollar])



Time_bars_1d = data1.groupby(pd.Grouper(key="timestamp", freq="1d")).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first' })
Time_bars_1d.columns = ['open','high','low','close','size','timestamp']
avg_dly_volume = np.mean(Time_bars_1d['size'])

avg_15m_volume = np.round(avg_dly_volume/96,decimals=-1)

avg_30m_volume = np.round(avg_dly_volume/48, decimals = -1)

avg_1h_volume = np.round(avg_dly_volume/24,decimals=-2)

avg_4h_volume = np.round(avg_dly_volume/5,decimals=-3)

avg_12h_volume = np.round(avg_dly_volume/2,decimals=-3)

avg_volumes = np.array([avg_15m_volume, avg_30m_volume, avg_1h_volume, avg_4h_volume, avg_12h_volume])

print(avg_volumes,'\n',avg_dollars)

In [ ]:
Volume_bars_50 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[0])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_50.columns = ['open','high','low','close','size','timestamp']
correlations.at['EOSUSDT','Volume-1'] = returns(Volume_bars_50['close']).autocorr(lag=1)


Volume_bars_100 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[1])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_100.columns = ['open','high','low','close','size','timestamp']
correlations.at['EOSUSDT','Volume-2'] = returns(Volume_bars_100['close']).autocorr(lag=1)


Volume_bars_500 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[2])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_500.columns = ['open','high','low','close','size','timestamp']
correlations.at['EOSUSDT','Volume-3'] = returns(Volume_bars_500['close']).autocorr(lag=1)


Volume_bars_1000 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[3])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_1000.columns = ['open','high','low','close','size','timestamp']
correlations.at['EOSUSDT','Volume-4'] = returns(Volume_bars_1000['close']).autocorr(lag=1)


Volume_bars_5000 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[4])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_5000.columns = ['open','high','low','close','size','timestamp']
correlations.at['EOSUSDT','Volume-5'] = returns(Volume_bars_5000['close']).autocorr(lag=1)



Dollar_bars_500 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[0])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_500.columns = ['open','high','low','close','size','timestamp']
correlations.at['EOSUSDT','Dollar-1'] = returns(Dollar_bars_500['close']).autocorr(lag=1)


Dollar_bars_1000 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[1])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_1000.columns = ['open','high','low','close','size','timestamp']
correlations.at['EOSUSDT','Dollar-2'] = returns(Dollar_bars_1000['close']).autocorr(lag=1)


Dollar_bars_10000 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[2])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_10000.columns = ['open','high','low','close','size','timestamp']
correlations.at['EOSUSDT','Dollar-3'] = returns(Dollar_bars_10000['close']).autocorr(lag=1)


Dollar_bars_100000 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[3])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_100000.columns = ['open','high','low','close','size','timestamp']
correlations.at['EOSUSDT','Dollar-4'] = returns(Dollar_bars_100000['close']).autocorr(lag=1)

Dollar_bars_1000000 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[4])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_1000000.columns = ['open','high','low','close','size','timestamp']
correlations.at['EOSUSDT','Dollar-5'] = returns(Dollar_bars_1000000['close']).autocorr(lag=1)


correlations


,Volume-1,Volume-2,Volume-3,Volume-4,Volume-5,Dollar-1,Dollar-2,Dollar-3,Dollar-4,Dollar-5
ADAUSDT,-0.0222421,-0.0162059,-0.0234384,-0.0403136,0.00702243,-0.047659,-0.00279578,0.0111431,-0.0662191,-0.0582422
BNBUSDT,-0.00430166,-0.00774128,-0.0157777,-0.0192799,-0.0799558,-0.121679,-0.0157654,-6.21017e-06,0.0296318,-0.0154122
EOSUSDT,0.0195058,0.0416292,0.08255,0.0926273,-0.025176,0.034728,0.000688956,-0.111252,-0.196515,-0.204495
DOTUSDT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ETHUSD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LINKUSDT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LTCUSD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
XTZUSDT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
XBTUSD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
XRPUSD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##DOT

In [ ]:
data1 = data[data.symbol == 'DOTUSDTH21']
# timestamp parsing
data1['timestamp'] = data1.timestamp.map(lambda t: datetime.strptime(t[:-3], "%Y-%m-%dD%H:%M:%S.%f"))

In [ ]:
Time_bars_1d = data1.groupby(pd.Grouper(key="timestamp", freq="1d")).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first', 'foreignNotional':'sum' })
Time_bars_1d.columns = ['open','high','low','close','size','timestamp','foreignNotional']
avg_dly_dollar = np.mean(Time_bars_1d['foreignNotional'])

avg_15m_dollar = np.round(avg_dly_dollar/96,decimals=-3)

avg_30m_dollar = np.round(avg_dly_dollar/48, decimals = -3)

avg_1h_dollar = np.round(avg_dly_dollar/24,decimals=-3)

avg_4h_dollar = np.round(avg_dly_dollar/5,decimals=-4)

avg_12h_dollar = np.round(avg_dly_dollar/2,decimals=-4)

avg_dollars = np.array([avg_15m_dollar, avg_30m_dollar, avg_1h_dollar, avg_4h_dollar, avg_12h_dollar])



Time_bars_1d = data1.groupby(pd.Grouper(key="timestamp", freq="1d")).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first' })
Time_bars_1d.columns = ['open','high','low','close','size','timestamp']
avg_dly_volume = np.mean(Time_bars_1d['size'])

avg_15m_volume = np.round(avg_dly_volume/96,decimals=-1)

avg_30m_volume = np.round(avg_dly_volume/48, decimals = -1)

avg_1h_volume = np.round(avg_dly_volume/24,decimals=-2)

avg_4h_volume = np.round(avg_dly_volume/5,decimals=-3)

avg_12h_volume = np.round(avg_dly_volume/2,decimals=-3)

avg_volumes = np.array([avg_15m_volume, avg_30m_volume, avg_1h_volume, avg_4h_volume, avg_12h_volume])

print(avg_volumes,'\n',avg_dollars)

In [ ]:
Volume_bars_50 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[0])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_50.columns = ['open','high','low','close','size','timestamp']
correlations.at['DOTUSDT','Volume-1'] = returns(Volume_bars_50['close']).autocorr(lag=1)


Volume_bars_100 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[1])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_100.columns = ['open','high','low','close','size','timestamp']
correlations.at['DOTUSDT','Volume-2'] = returns(Volume_bars_100['close']).autocorr(lag=1)


Volume_bars_500 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[2])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_500.columns = ['open','high','low','close','size','timestamp']
correlations.at['DOTUSDT','Volume-3'] = returns(Volume_bars_500['close']).autocorr(lag=1)


Volume_bars_1000 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[3])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_1000.columns = ['open','high','low','close','size','timestamp']
correlations.at['DOTUSDT','Volume-4'] = returns(Volume_bars_1000['close']).autocorr(lag=1)


Volume_bars_5000 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[4])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_5000.columns = ['open','high','low','close','size','timestamp']
correlations.at['DOTUSDT','Volume-5'] = returns(Volume_bars_5000['close']).autocorr(lag=1)



Dollar_bars_500 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[0])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_500.columns = ['open','high','low','close','size','timestamp']
correlations.at['DOTUSDT','Dollar-1'] = returns(Dollar_bars_500['close']).autocorr(lag=1)


Dollar_bars_1000 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[1])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_1000.columns = ['open','high','low','close','size','timestamp']
correlations.at['DOTUSDT','Dollar-2'] = returns(Dollar_bars_1000['close']).autocorr(lag=1)


Dollar_bars_10000 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[2])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_10000.columns = ['open','high','low','close','size','timestamp']
correlations.at['DOTUSDT','Dollar-3'] = returns(Dollar_bars_10000['close']).autocorr(lag=1)


Dollar_bars_100000 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[3])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_100000.columns = ['open','high','low','close','size','timestamp']
correlations.at['DOTUSDT','Dollar-4'] = returns(Dollar_bars_100000['close']).autocorr(lag=1)

Dollar_bars_1000000 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[4])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_1000000.columns = ['open','high','low','close','size','timestamp']
correlations.at['DOTUSDT','Dollar-5'] = returns(Dollar_bars_1000000['close']).autocorr(lag=1)


correlations


,Volume-1,Volume-2,Volume-3,Volume-4,Volume-5,Dollar-1,Dollar-2,Dollar-3,Dollar-4,Dollar-5
ADAUSDT,-0.0222421,-0.0162059,-0.0234384,-0.0403136,0.00702243,-0.047659,-0.00279578,0.0111431,-0.0662191,-0.0582422
BNBUSDT,-0.00430166,-0.00774128,-0.0157777,-0.0192799,-0.0799558,-0.121679,-0.0157654,-6.21017e-06,0.0296318,-0.0154122
EOSUSDT,0.0195058,0.0416292,0.08255,0.0926273,-0.025176,0.034728,0.000688956,-0.111252,-0.196515,-0.204495
DOTUSDT,-0.111909,-0.110287,-0.0290842,-0.0866501,0.0496966,-0.027191,0.025302,-0.0246733,0.135908,-0.059194
ETHUSD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LINKUSDT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LTCUSD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
XTZUSDT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
XBTUSD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
XRPUSD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##ETH

In [ ]:
data1 = data[data.symbol == 'ETHUSD']

# timestamp parsing
data1['timestamp'] = data1.timestamp.map(lambda t: datetime.strptime(t[:-3], "%Y-%m-%dD%H:%M:%S.%f"))

In [ ]:
Time_bars_1d = data1.groupby(pd.Grouper(key="timestamp", freq="1d")).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first', 'foreignNotional':'sum' })
Time_bars_1d.columns = ['open','high','low','close','size','timestamp','foreignNotional']
avg_dly_dollar = np.mean(Time_bars_1d['foreignNotional'])

avg_15m_dollar = np.round(avg_dly_dollar/96,decimals=-3)

avg_30m_dollar = np.round(avg_dly_dollar/48, decimals = -3)

avg_1h_dollar = np.round(avg_dly_dollar/24,decimals=-3)

avg_4h_dollar = np.round(avg_dly_dollar/5,decimals=-4)

avg_12h_dollar = np.round(avg_dly_dollar/2,decimals=-4)

avg_dollars = np.array([avg_15m_dollar, avg_30m_dollar, avg_1h_dollar, avg_4h_dollar, avg_12h_dollar])



Time_bars_1d = data1.groupby(pd.Grouper(key="timestamp", freq="1d")).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first' })
Time_bars_1d.columns = ['open','high','low','close','size','timestamp']
avg_dly_volume = np.mean(Time_bars_1d['size'])

avg_15m_volume = np.round(avg_dly_volume/96,decimals=-1)

avg_30m_volume = np.round(avg_dly_volume/48, decimals = -1)

avg_1h_volume = np.round(avg_dly_volume/24,decimals=-2)

avg_4h_volume = np.round(avg_dly_volume/5,decimals=-3)

avg_12h_volume = np.round(avg_dly_volume/2,decimals=-3)

avg_volumes = np.array([avg_15m_volume, avg_30m_volume, avg_1h_volume, avg_4h_volume, avg_12h_volume])

print(avg_volumes,'\n',avg_dollars)

In [ ]:
Volume_bars_50 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[0])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_50.columns = ['open','high','low','close','size','timestamp']
correlations.at['ETHUSD','Volume-1'] = returns(Volume_bars_50['close']).autocorr(lag=1)


Volume_bars_100 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[1])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_100.columns = ['open','high','low','close','size','timestamp']
correlations.at['ETHUSD','Volume-2'] = returns(Volume_bars_100['close']).autocorr(lag=1)


Volume_bars_500 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[2])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_500.columns = ['open','high','low','close','size','timestamp']
correlations.at['ETHUSD','Volume-3'] = returns(Volume_bars_500['close']).autocorr(lag=1)


Volume_bars_1000 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[3])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_1000.columns = ['open','high','low','close','size','timestamp']
correlations.at['ETHUSD','Volume-4'] = returns(Volume_bars_1000['close']).autocorr(lag=1)


Volume_bars_5000 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[4])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_5000.columns = ['open','high','low','close','size','timestamp']
correlations.at['ETHUSD','Volume-5'] = returns(Volume_bars_5000['close']).autocorr(lag=1)



Dollar_bars_500 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[0])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_500.columns = ['open','high','low','close','size','timestamp']
correlations.at['ETHUSD','Dollar-1'] = returns(Dollar_bars_500['close']).autocorr(lag=1)


Dollar_bars_1000 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[1])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_1000.columns = ['open','high','low','close','size','timestamp']
correlations.at['ETHUSD','Dollar-2'] = returns(Dollar_bars_1000['close']).autocorr(lag=1)


Dollar_bars_10000 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[2])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_10000.columns = ['open','high','low','close','size','timestamp']
correlations.at['ETHUSD','Dollar-3'] = returns(Dollar_bars_10000['close']).autocorr(lag=1)


Dollar_bars_100000 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[3])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_100000.columns = ['open','high','low','close','size','timestamp']
correlations.at['ETHUSD','Dollar-4'] = returns(Dollar_bars_100000['close']).autocorr(lag=1)

Dollar_bars_1000000 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[4])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_1000000.columns = ['open','high','low','close','size','timestamp']
correlations.at['ETHUSD','Dollar-5'] = returns(Dollar_bars_1000000['close']).autocorr(lag=1)


correlations


,Volume-1,Volume-2,Volume-3,Volume-4,Volume-5,Dollar-1,Dollar-2,Dollar-3,Dollar-4,Dollar-5
ADAUSDT,-0.0222421,-0.0162059,-0.0234384,-0.0403136,0.00702243,-0.047659,-0.00279578,0.0111431,-0.0662191,-0.0582422
BNBUSDT,-0.00430166,-0.00774128,-0.0157777,-0.0192799,-0.0799558,-0.121679,-0.0157654,-6.21017e-06,0.0296318,-0.0154122
EOSUSDT,0.0195058,0.0416292,0.08255,0.0926273,-0.025176,0.034728,0.000688956,-0.111252,-0.196515,-0.204495
DOTUSDT,-0.111909,-0.110287,-0.0290842,-0.0866501,0.0496966,-0.027191,0.025302,-0.0246733,0.135908,-0.059194
ETHUSD,-0.0228118,0.00674662,0.00755689,0.0173865,0.0120125,0.0177467,0.0160131,0.0116154,0.0124786,0.011786
LINKUSDT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LTCUSD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
XTZUSDT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
XBTUSD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
XRPUSD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##LINK

In [ ]:
data1 = data[data.symbol == 'LINKUSDT'] 

# timestamp parsing
data1['timestamp'] = data1.timestamp.map(lambda t: datetime.strptime(t[:-3], "%Y-%m-%dD%H:%M:%S.%f"))

In [ ]:
Time_bars_1d = data1.groupby(pd.Grouper(key="timestamp", freq="1d")).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first', 'foreignNotional':'sum' })
Time_bars_1d.columns = ['open','high','low','close','size','timestamp','foreignNotional']
avg_dly_dollar = np.mean(Time_bars_1d['foreignNotional'])

avg_15m_dollar = np.round(avg_dly_dollar/96,decimals=-3)

avg_30m_dollar = np.round(avg_dly_dollar/48, decimals = -3)

avg_1h_dollar = np.round(avg_dly_dollar/24,decimals=-3)

avg_4h_dollar = np.round(avg_dly_dollar/5,decimals=-4)

avg_12h_dollar = np.round(avg_dly_dollar/2,decimals=-4)

avg_dollars = np.array([avg_15m_dollar, avg_30m_dollar, avg_1h_dollar, avg_4h_dollar, avg_12h_dollar])



Time_bars_1d = data1.groupby(pd.Grouper(key="timestamp", freq="1d")).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first' })
Time_bars_1d.columns = ['open','high','low','close','size','timestamp']
avg_dly_volume = np.mean(Time_bars_1d['size'])

avg_15m_volume = np.round(avg_dly_volume/96,decimals=-1)

avg_30m_volume = np.round(avg_dly_volume/48, decimals = -1)

avg_1h_volume = np.round(avg_dly_volume/24,decimals=-2)

avg_4h_volume = np.round(avg_dly_volume/5,decimals=-3)

avg_12h_volume = np.round(avg_dly_volume/2,decimals=-3)

avg_volumes = np.array([avg_15m_volume, avg_30m_volume, avg_1h_volume, avg_4h_volume, avg_12h_volume])

print(avg_volumes,'\n',avg_dollars)

In [ ]:
Volume_bars_50 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[0])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_50.columns = ['open','high','low','close','size','timestamp']
correlations.at['LINKUSDT','Volume-1'] = returns(Volume_bars_50['close']).autocorr(lag=1)


Volume_bars_100 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[1])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_100.columns = ['open','high','low','close','size','timestamp']
correlations.at['LINKUSDT','Volume-2'] = returns(Volume_bars_100['close']).autocorr(lag=1)


Volume_bars_500 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[2])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_500.columns = ['open','high','low','close','size','timestamp']
correlations.at['LINKUSDT','Volume-3'] = returns(Volume_bars_500['close']).autocorr(lag=1)


Volume_bars_1000 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[3])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_1000.columns = ['open','high','low','close','size','timestamp']
correlations.at['LINKUSDT','Volume-4'] = returns(Volume_bars_1000['close']).autocorr(lag=1)


Volume_bars_5000 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[4])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_5000.columns = ['open','high','low','close','size','timestamp']
correlations.at['LINKUSDT','Volume-5'] = returns(Volume_bars_5000['close']).autocorr(lag=1)



Dollar_bars_500 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[0])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_500.columns = ['open','high','low','close','size','timestamp']
correlations.at['LINKUSDT','Dollar-1'] = returns(Dollar_bars_500['close']).autocorr(lag=1)


Dollar_bars_1000 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[1])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_1000.columns = ['open','high','low','close','size','timestamp']
correlations.at['LINKUSDT','Dollar-2'] = returns(Dollar_bars_1000['close']).autocorr(lag=1)


Dollar_bars_10000 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[2])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_10000.columns = ['open','high','low','close','size','timestamp']
correlations.at['LINKUSDT','Dollar-3'] = returns(Dollar_bars_10000['close']).autocorr(lag=1)


Dollar_bars_100000 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[3])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_100000.columns = ['open','high','low','close','size','timestamp']
correlations.at['LINKUSDT','Dollar-4'] = returns(Dollar_bars_100000['close']).autocorr(lag=1)

Dollar_bars_1000000 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[4])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_1000000.columns = ['open','high','low','close','size','timestamp']
correlations.at['LINKUSDT','Dollar-5'] = returns(Dollar_bars_1000000['close']).autocorr(lag=1)


correlations


,Volume-1,Volume-2,Volume-3,Volume-4,Volume-5,Dollar-1,Dollar-2,Dollar-3,Dollar-4,Dollar-5
ADAUSDT,-0.0222421,-0.0162059,-0.0234384,-0.0403136,0.00702243,-0.047659,-0.00279578,0.0111431,-0.0662191,-0.0582422
BNBUSDT,-0.00430166,-0.00774128,-0.0157777,-0.0192799,-0.0799558,-0.121679,-0.0157654,-6.21017e-06,0.0296318,-0.0154122
EOSUSDT,0.0195058,0.0416292,0.08255,0.0926273,-0.025176,0.034728,0.000688956,-0.111252,-0.196515,-0.204495
DOTUSDT,-0.111909,-0.110287,-0.0290842,-0.0866501,0.0496966,-0.027191,0.025302,-0.0246733,0.135908,-0.059194
ETHUSD,-0.0228118,0.00674662,0.00755689,0.0173865,0.0120125,0.0177467,0.0160131,0.0116154,0.0124786,0.011786
LINKUSDT,-0.0026451,-0.016928,-0.011201,0.00829243,0.0820841,-0.00988345,-0.0106066,-0.00585707,0.00344529,0.098384
LTCUSD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
XTZUSDT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
XBTUSD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
XRPUSD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##LTC

In [ ]:
data1 = data[data.symbol == 'LTCUSD']

# timestamp parsing 
data1['timestamp'] = data1.timestamp.map(lambda t: datetime.strptime(t[:-3], "%Y-%m-%dD%H:%M:%S.%f"))

In [ ]:
Time_bars_1d = data1.groupby(pd.Grouper(key="timestamp", freq="1d")).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first', 'foreignNotional':'sum' })
Time_bars_1d.columns = ['open','high','low','close','size','timestamp','foreignNotional']
avg_dly_dollar = np.mean(Time_bars_1d['foreignNotional'])

avg_15m_dollar = np.round(avg_dly_dollar/96,decimals=-3)

avg_30m_dollar = np.round(avg_dly_dollar/48, decimals = -3)

avg_1h_dollar = np.round(avg_dly_dollar/24,decimals=-3)

avg_4h_dollar = np.round(avg_dly_dollar/5,decimals=-4)

avg_12h_dollar = np.round(avg_dly_dollar/2,decimals=-4)

avg_dollars = np.array([avg_15m_dollar, avg_30m_dollar, avg_1h_dollar, avg_4h_dollar, avg_12h_dollar])



Time_bars_1d = data1.groupby(pd.Grouper(key="timestamp", freq="1d")).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first' })
Time_bars_1d.columns = ['open','high','low','close','size','timestamp']
avg_dly_volume = np.mean(Time_bars_1d['size'])

avg_15m_volume = np.round(avg_dly_volume/96,decimals=-1)

avg_30m_volume = np.round(avg_dly_volume/48, decimals = -1)

avg_1h_volume = np.round(avg_dly_volume/24,decimals=-2)

avg_4h_volume = np.round(avg_dly_volume/5,decimals=-3)

avg_12h_volume = np.round(avg_dly_volume/2,decimals=-3)

avg_volumes = np.array([avg_15m_volume, avg_30m_volume, avg_1h_volume, avg_4h_volume, avg_12h_volume])

print(avg_volumes,'\n',avg_dollars)

In [ ]:
Volume_bars_50 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[0])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_50.columns = ['open','high','low','close','size','timestamp']
correlations.at['LTCUSD','Volume-1'] = returns(Volume_bars_50['close']).autocorr(lag=1)


Volume_bars_100 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[1])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_100.columns = ['open','high','low','close','size','timestamp']
correlations.at['LTCUSD','Volume-2'] = returns(Volume_bars_100['close']).autocorr(lag=1)


Volume_bars_500 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[2])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_500.columns = ['open','high','low','close','size','timestamp']
correlations.at['LTCUSD','Volume-3'] = returns(Volume_bars_500['close']).autocorr(lag=1)


Volume_bars_1000 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[3])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_1000.columns = ['open','high','low','close','size','timestamp']
correlations.at['LTCUSD','Volume-4'] = returns(Volume_bars_1000['close']).autocorr(lag=1)


Volume_bars_5000 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[4])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_5000.columns = ['open','high','low','close','size','timestamp']
correlations.at['LTCUSD','Volume-5'] = returns(Volume_bars_5000['close']).autocorr(lag=1)



Dollar_bars_500 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[0])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_500.columns = ['open','high','low','close','size','timestamp']
correlations.at['LTCUSD','Dollar-1'] = returns(Dollar_bars_500['close']).autocorr(lag=1)


Dollar_bars_1000 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[1])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_1000.columns = ['open','high','low','close','size','timestamp']
correlations.at['LTCUSD','Dollar-2'] = returns(Dollar_bars_1000['close']).autocorr(lag=1)


Dollar_bars_10000 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[2])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_10000.columns = ['open','high','low','close','size','timestamp']
correlations.at['LTCUSD','Dollar-3'] = returns(Dollar_bars_10000['close']).autocorr(lag=1)


Dollar_bars_100000 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[3])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_100000.columns = ['open','high','low','close','size','timestamp']
correlations.at['LTCUSD','Dollar-4'] = returns(Dollar_bars_100000['close']).autocorr(lag=1)

Dollar_bars_1000000 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[4])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_1000000.columns = ['open','high','low','close','size','timestamp']
correlations.at['LTCUSD','Dollar-5'] = returns(Dollar_bars_1000000['close']).autocorr(lag=1)


correlations


,Volume-1,Volume-2,Volume-3,Volume-4,Volume-5,Dollar-1,Dollar-2,Dollar-3,Dollar-4,Dollar-5
ADAUSDT,-0.0222421,-0.0162059,-0.0234384,-0.0403136,0.00702243,-0.047659,-0.00279578,0.0111431,-0.0662191,-0.0582422
BNBUSDT,-0.00430166,-0.00774128,-0.0157777,-0.0192799,-0.0799558,-0.121679,-0.0157654,-6.21017e-06,0.0296318,-0.0154122
EOSUSDT,0.0195058,0.0416292,0.08255,0.0926273,-0.025176,0.034728,0.000688956,-0.111252,-0.196515,-0.204495
DOTUSDT,-0.111909,-0.110287,-0.0290842,-0.0866501,0.0496966,-0.027191,0.025302,-0.0246733,0.135908,-0.059194
ETHUSD,-0.0228118,0.00674662,0.00755689,0.0173865,0.0120125,0.0177467,0.0160131,0.0116154,0.0124786,0.011786
LINKUSDT,-0.0026451,-0.016928,-0.011201,0.00829243,0.0820841,-0.00988345,-0.0106066,-0.00585707,0.00344529,0.098384
LTCUSD,-0.0437398,-0.0302064,-0.0128855,0.00545261,0.00572731,-0.00241765,-0.0036416,-0.038919,0.0134392,-0.0164666
XTZUSDT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
XBTUSD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
XRPUSD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
correlations = correlations.rename(index={'SOLUSDT': 'LTCUSD'})

##XTZ

In [ ]:
data1 = data[data.symbol == 'XTZUSDTH21']
# timestamp parsing 
data1['timestamp'] = data1.timestamp.map(lambda t: datetime.strptime(t[:-3], "%Y-%m-%dD%H:%M:%S.%f"))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
Time_bars_1d = data1.groupby(pd.Grouper(key="timestamp", freq="1d")).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first', 'foreignNotional':'sum' })
Time_bars_1d.columns = ['open','high','low','close','size','timestamp','foreignNotional']
avg_dly_dollar = np.mean(Time_bars_1d['foreignNotional'])

avg_15m_dollar = np.round(avg_dly_dollar/96,decimals=-3)

avg_30m_dollar = np.round(avg_dly_dollar/48, decimals = -3)

avg_1h_dollar = np.round(avg_dly_dollar/24,decimals=-3)

avg_4h_dollar = np.round(avg_dly_dollar/5,decimals=-4)

avg_12h_dollar = np.round(avg_dly_dollar/2,decimals=-4)

avg_dollars = np.array([avg_15m_dollar, avg_30m_dollar, avg_1h_dollar, avg_4h_dollar, avg_12h_dollar])



Time_bars_1d = data1.groupby(pd.Grouper(key="timestamp", freq="1d")).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first' })
Time_bars_1d.columns = ['open','high','low','close','size','timestamp']
avg_dly_volume = np.mean(Time_bars_1d['size'])

avg_15m_volume = np.round(avg_dly_volume/96,decimals=-1)

avg_30m_volume = np.round(avg_dly_volume/48, decimals = -1)

avg_1h_volume = np.round(avg_dly_volume/24,decimals=-2)

avg_4h_volume = np.round(avg_dly_volume/5,decimals=-3)

avg_12h_volume = np.round(avg_dly_volume/2,decimals=-3)

avg_volumes = np.array([avg_15m_volume, avg_30m_volume, avg_1h_volume, avg_4h_volume, avg_12h_volume])

print(avg_volumes,'\n',avg_dollars)

In [ ]:
Volume_bars_50 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[0])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_50.columns = ['open','high','low','close','size','timestamp']
correlations.at['XTZUSDT','Volume-1'] = returns(Volume_bars_50['close']).autocorr(lag=1)


Volume_bars_100 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[1])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_100.columns = ['open','high','low','close','size','timestamp']
correlations.at['XTZUSDT','Volume-2'] = returns(Volume_bars_100['close']).autocorr(lag=1)


Volume_bars_500 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[2])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_500.columns = ['open','high','low','close','size','timestamp']
correlations.at['XTZUSDT','Volume-3'] = returns(Volume_bars_500['close']).autocorr(lag=1)


Volume_bars_1000 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[3])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_1000.columns = ['open','high','low','close','size','timestamp']
correlations.at['XTZUSDT','Volume-4'] = returns(Volume_bars_1000['close']).autocorr(lag=1)


Volume_bars_5000 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[4])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_5000.columns = ['open','high','low','close','size','timestamp']
correlations.at['XTZUSDT','Volume-5'] = returns(Volume_bars_5000['close']).autocorr(lag=1)



Dollar_bars_500 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[0])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_500.columns = ['open','high','low','close','size','timestamp']
correlations.at['XTZUSDT','Dollar-1'] = returns(Dollar_bars_500['close']).autocorr(lag=1)


Dollar_bars_1000 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[1])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_1000.columns = ['open','high','low','close','size','timestamp']
correlations.at['XTZUSDT','Dollar-2'] = returns(Dollar_bars_1000['close']).autocorr(lag=1)


Dollar_bars_10000 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[2])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_10000.columns = ['open','high','low','close','size','timestamp']
correlations.at['XTZUSDT','Dollar-3'] = returns(Dollar_bars_10000['close']).autocorr(lag=1)


Dollar_bars_100000 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[3])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_100000.columns = ['open','high','low','close','size','timestamp']
correlations.at['XTZUSDT','Dollar-4'] = returns(Dollar_bars_100000['close']).autocorr(lag=1)

Dollar_bars_1000000 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[4])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_1000000.columns = ['open','high','low','close','size','timestamp']
correlations.at['XTZUSDT','Dollar-5'] = returns(Dollar_bars_1000000['close']).autocorr(lag=1)


correlations


##YFI

In [ ]:
data1 = data[data.symbol == 'YFIUSDTH21']
# timestamp parsing 
data1['timestamp'] = data1.timestamp.map(lambda t: datetime.strptime(t[:-3], "%Y-%m-%dD%H:%M:%S.%f"))

In [ ]:
correlations.append(pd.Series(name='YFIUSDT'))

In [ ]:
Time_bars_1d = data1.groupby(pd.Grouper(key="timestamp", freq="1d")).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first', 'foreignNotional':'sum' })
Time_bars_1d.columns = ['open','high','low','close','size','timestamp','foreignNotional']
avg_dly_dollar = np.mean(Time_bars_1d['foreignNotional'])

avg_15m_dollar = np.round(avg_dly_dollar/96,decimals=-3)

avg_30m_dollar = np.round(avg_dly_dollar/48, decimals = -3)

avg_1h_dollar = np.round(avg_dly_dollar/24,decimals=-3)

avg_4h_dollar = np.round(avg_dly_dollar/5,decimals=-4)

avg_12h_dollar = np.round(avg_dly_dollar/2,decimals=-4)

avg_dollars = np.array([avg_15m_dollar, avg_30m_dollar, avg_1h_dollar, avg_4h_dollar, avg_12h_dollar])



Time_bars_1d = data1.groupby(pd.Grouper(key="timestamp", freq="1d")).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first' })
Time_bars_1d.columns = ['open','high','low','close','size','timestamp']
avg_dly_volume = np.mean(Time_bars_1d['size'])

avg_15m_volume = np.round(avg_dly_volume/96,decimals=-1)

avg_30m_volume = np.round(avg_dly_volume/48, decimals = -1)

avg_1h_volume = np.round(avg_dly_volume/24,decimals=-2)

avg_4h_volume = np.round(avg_dly_volume/5,decimals=-3)

avg_12h_volume = np.round(avg_dly_volume/2,decimals=-3)

avg_volumes = np.array([avg_15m_volume, avg_30m_volume, avg_1h_volume, avg_4h_volume, avg_12h_volume])

print(avg_volumes,'\n',avg_dollars)

In [ ]:
Volume_bars_50 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[0])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_50.columns = ['open','high','low','close','size','timestamp']
correlations.at['YFIUSDT','Volume-1'] = returns(Volume_bars_50['close']).autocorr(lag=1)


Volume_bars_100 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[1])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_100.columns = ['open','high','low','close','size','timestamp']
correlations.at['YFIUSDT','Volume-2'] = returns(Volume_bars_100['close']).autocorr(lag=1)


Volume_bars_500 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[2])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_500.columns = ['open','high','low','close','size','timestamp']
correlations.at['YFIUSDT','Volume-3'] = returns(Volume_bars_500['close']).autocorr(lag=1)


Volume_bars_1000 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[3])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_1000.columns = ['open','high','low','close','size','timestamp']
correlations.at['YFIUSDT','Volume-4'] = returns(Volume_bars_1000['close']).autocorr(lag=1)


Volume_bars_5000 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[4])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_5000.columns = ['open','high','low','close','size','timestamp']
correlations.at['YFIUSDT','Volume-5'] = returns(Volume_bars_5000['close']).autocorr(lag=1)



Dollar_bars_500 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[0])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_500.columns = ['open','high','low','close','size','timestamp']
correlations.at['YFIUSDT','Dollar-1'] = returns(Dollar_bars_500['close']).autocorr(lag=1)


Dollar_bars_1000 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[1])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_1000.columns = ['open','high','low','close','size','timestamp']
correlations.at['YFIUSDT','Dollar-2'] = returns(Dollar_bars_1000['close']).autocorr(lag=1)


Dollar_bars_10000 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[2])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_10000.columns = ['open','high','low','close','size','timestamp']
correlations.at['YFIUSDT','Dollar-3'] = returns(Dollar_bars_10000['close']).autocorr(lag=1)


Dollar_bars_100000 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[3])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_100000.columns = ['open','high','low','close','size','timestamp']
correlations.at['YFIUSDT','Dollar-4'] = returns(Dollar_bars_100000['close']).autocorr(lag=1)

Dollar_bars_1000000 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[4])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_1000000.columns = ['open','high','low','close','size','timestamp']
correlations.at['YFIUSDT','Dollar-5'] = returns(Dollar_bars_1000000['close']).autocorr(lag=1)


correlations


##XBT

In [ ]:
data1 = data[data.symbol == 'XBTUSD']
# timestamp parsing 
data1['timestamp'] = data1.timestamp.map(lambda t: datetime.strptime(t[:-3], "%Y-%m-%dD%H:%M:%S.%f"))

In [ ]:
Time_bars_1d = data1.groupby(pd.Grouper(key="timestamp", freq="1d")).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first', 'foreignNotional':'sum' })
Time_bars_1d.columns = ['open','high','low','close','size','timestamp','foreignNotional']
avg_dly_dollar = np.mean(Time_bars_1d['foreignNotional'])

avg_15m_dollar = np.round(avg_dly_dollar/96,decimals=-3)

avg_30m_dollar = np.round(avg_dly_dollar/48, decimals = -3)

avg_1h_dollar = np.round(avg_dly_dollar/24,decimals=-3)

avg_4h_dollar = np.round(avg_dly_dollar/5,decimals=-4)

avg_12h_dollar = np.round(avg_dly_dollar/2,decimals=-4)

avg_dollars = np.array([avg_15m_dollar, avg_30m_dollar, avg_1h_dollar, avg_4h_dollar, avg_12h_dollar])



Time_bars_1d = data1.groupby(pd.Grouper(key="timestamp", freq="1d")).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first' })
Time_bars_1d.columns = ['open','high','low','close','size','timestamp']
avg_dly_volume = np.mean(Time_bars_1d['size'])

avg_15m_volume = np.round(avg_dly_volume/96,decimals=-1)

avg_30m_volume = np.round(avg_dly_volume/48, decimals = -1)

avg_1h_volume = np.round(avg_dly_volume/24,decimals=-2)

avg_4h_volume = np.round(avg_dly_volume/5,decimals=-3)

avg_12h_volume = np.round(avg_dly_volume/2,decimals=-3)

avg_volumes = np.array([avg_15m_volume, avg_30m_volume, avg_1h_volume, avg_4h_volume, avg_12h_volume])

print(avg_volumes,'\n',avg_dollars)

In [ ]:
Volume_bars_50 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[0])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_50.columns = ['open','high','low','close','size','timestamp']
correlations.at['XBTUSD','Volume-1'] = returns(Volume_bars_50['close']).autocorr(lag=1)


Volume_bars_100 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[1])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_100.columns = ['open','high','low','close','size','timestamp']
correlations.at['XBTUSD','Volume-2'] = returns(Volume_bars_100['close']).autocorr(lag=1)


Volume_bars_500 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[2])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_500.columns = ['open','high','low','close','size','timestamp']
correlations.at['XBTUSD','Volume-3'] = returns(Volume_bars_500['close']).autocorr(lag=1)


Volume_bars_1000 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[3])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_1000.columns = ['open','high','low','close','size','timestamp']
correlations.at['XBTUSD','Volume-4'] = returns(Volume_bars_1000['close']).autocorr(lag=1)


Volume_bars_5000 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[4])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_5000.columns = ['open','high','low','close','size','timestamp']
correlations.at['XBTUSD','Volume-5'] = returns(Volume_bars_5000['close']).autocorr(lag=1)



Dollar_bars_500 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[0])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_500.columns = ['open','high','low','close','size','timestamp']
correlations.at['XBTUSD','Dollar-1'] = returns(Dollar_bars_500['close']).autocorr(lag=1)


Dollar_bars_1000 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[1])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_1000.columns = ['open','high','low','close','size','timestamp']
correlations.at['XBTUSD','Dollar-2'] = returns(Dollar_bars_1000['close']).autocorr(lag=1)


Dollar_bars_10000 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[2])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_10000.columns = ['open','high','low','close','size','timestamp']
correlations.at['XBTUSD','Dollar-3'] = returns(Dollar_bars_10000['close']).autocorr(lag=1)


Dollar_bars_100000 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[3])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_100000.columns = ['open','high','low','close','size','timestamp']
correlations.at['XBTUSD','Dollar-4'] = returns(Dollar_bars_100000['close']).autocorr(lag=1)

Dollar_bars_1000000 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[4])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_1000000.columns = ['open','high','low','close','size','timestamp']
correlations.at['XBTUSD','Dollar-5'] = returns(Dollar_bars_1000000['close']).autocorr(lag=1)


correlations


##XRP

In [ ]:
data1 = data[data.symbol == 'XRPUSD']
# timestamp parsing 
data1['timestamp'] = data1.timestamp.map(lambda t: datetime.strptime(t[:-3], "%Y-%m-%dD%H:%M:%S.%f"))

In [ ]:
Time_bars_1d = data1.groupby(pd.Grouper(key="timestamp", freq="1d")).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first', 'foreignNotional':'sum' })
Time_bars_1d.columns = ['open','high','low','close','size','timestamp','foreignNotional']
avg_dly_dollar = np.mean(Time_bars_1d['foreignNotional'])

avg_15m_dollar = np.round(avg_dly_dollar/96,decimals=-3)

avg_30m_dollar = np.round(avg_dly_dollar/48, decimals = -3)

avg_1h_dollar = np.round(avg_dly_dollar/24,decimals=-3)

avg_4h_dollar = np.round(avg_dly_dollar/5,decimals=-4)

avg_12h_dollar = np.round(avg_dly_dollar/2,decimals=-4)

avg_dollars = np.array([avg_15m_dollar, avg_30m_dollar, avg_1h_dollar, avg_4h_dollar, avg_12h_dollar])



Time_bars_1d = data1.groupby(pd.Grouper(key="timestamp", freq="1d")).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first' })
Time_bars_1d.columns = ['open','high','low','close','size','timestamp']
avg_dly_volume = np.mean(Time_bars_1d['size'])

avg_15m_volume = np.round(avg_dly_volume/96,decimals=-1)

avg_30m_volume = np.round(avg_dly_volume/48, decimals = -1)

avg_1h_volume = np.round(avg_dly_volume/24,decimals=-2)

avg_4h_volume = np.round(avg_dly_volume/5,decimals=-3)

avg_12h_volume = np.round(avg_dly_volume/2,decimals=-3)

avg_volumes = np.array([avg_15m_volume, avg_30m_volume, avg_1h_volume, avg_4h_volume, avg_12h_volume])

print(avg_volumes,'\n',avg_dollars)

In [ ]:
Volume_bars_50 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[0])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_50.columns = ['open','high','low','close','size','timestamp']
correlations.at['XRPUSD','Volume-1'] = returns(Volume_bars_50['close']).autocorr(lag=1)


Volume_bars_100 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[1])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_100.columns = ['open','high','low','close','size','timestamp']
correlations.at['XRPUSD','Volume-2'] = returns(Volume_bars_100['close']).autocorr(lag=1)


Volume_bars_500 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[2])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_500.columns = ['open','high','low','close','size','timestamp']
correlations.at['XRPUSD','Volume-3'] = returns(Volume_bars_500['close']).autocorr(lag=1)


Volume_bars_1000 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[3])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_1000.columns = ['open','high','low','close','size','timestamp']
correlations.at['XRPUSD','Volume-4'] = returns(Volume_bars_1000['close']).autocorr(lag=1)


Volume_bars_5000 = data1.groupby(bar(np.cumsum(data1['size']), avg_volumes[4])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Volume_bars_5000.columns = ['open','high','low','close','size','timestamp']
correlations.at['XRPUSD','Volume-5'] = returns(Volume_bars_5000['close']).autocorr(lag=1)



Dollar_bars_500 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[0])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_500.columns = ['open','high','low','close','size','timestamp']
correlations.at['XRPUSD','Dollar-1'] = returns(Dollar_bars_500['close']).autocorr(lag=1)


Dollar_bars_1000 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[1])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_1000.columns = ['open','high','low','close','size','timestamp']
correlations.at['XRPUSD','Dollar-2'] = returns(Dollar_bars_1000['close']).autocorr(lag=1)


Dollar_bars_10000 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[2])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_10000.columns = ['open','high','low','close','size','timestamp']
correlations.at['XRPUSD','Dollar-3'] = returns(Dollar_bars_10000['close']).autocorr(lag=1)


Dollar_bars_100000 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[3])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_100000.columns = ['open','high','low','close','size','timestamp']
correlations.at['XRPUSD','Dollar-4'] = returns(Dollar_bars_100000['close']).autocorr(lag=1)

Dollar_bars_1000000 = data1.groupby(bar(np.cumsum(data1['foreignNotional']), avg_dollars[4])).agg({'price': 'ohlc', 'size': 'sum', 'timestamp': 'first'})
Dollar_bars_1000000.columns = ['open','high','low','close','size','timestamp']
correlations.at['XRPUSD','Dollar-5'] = returns(Dollar_bars_1000000['close']).autocorr(lag=1)


correlations


In [ ]:
correlations.loc['mean'] = correlations.mean()
correlations

In [ ]:
correlations.drop(['DOGEUSDT','XLMUSDT','TRXUSDT','UNIUSDT'],inplace=True)
correlations

In [ ]:
row_names = ['ADAUSDT','BNBUSDT','DOTUSDT','ETHUSD','LINKUSDT','LTCUSD','XBTUSD','XRPUSD','EOSUSDT','XTZUSDT','YFIUSDT','Mean']


cormat = np.array(correlations,dtype='float64')

fig, ax = plt.subplots(figsize=(15,15))         # Sample figsize in inches
corrmat =  sns.heatmap(cormat, annot = True, linewidths=.5, ax=ax, xticklabels=column_names,
                       yticklabels=row_names, cmap="Blues")

ax.xaxis.set_ticks_position('top')


#Correlation final matrix


In [ ]:
from google.colab import files
fig = corrmat.get_figure()
fig.savefig('Correlation_volume_dollar.png')
